In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

# Tải mô hình và bộ phát hiện khuôn mặt
try:
    model = load_model("emotion_recognition_model.keras")
except Exception as e:
    print(f"Không thể tải mô hình: {e}")
    exit(1)

emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

try:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    if face_cascade.empty():
        raise ValueError("Không thể tải file Haar Cascade.")
except Exception as e:
    print(f"Không thể tải Haar Cascade: {e}")
    exit(1)

# Hàm dự đoán cảm xúc tối ưu
@tf.function(reduce_retracing=True)
def predict_emotion_once(input_img):
    return model(input_img, training=False)

# Khởi tạo webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Không thể mở webcam. Vui lòng kiểm tra thiết bị.")
    exit(1)

# Thiết lập kích thước khung hình
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc khung hình từ webcam.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float32") / 255.0
        roi = roi.reshape(1, 48, 48, 1)

        prediction = predict_emotion_once(roi)
        label = emotion_labels[np.argmax(prediction)]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Hiển thị thông báo nếu không tìm thấy khuôn mặt
    if len(faces) == 0:
        cv2.putText(frame, "No face detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Emotion Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()